In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import timeit
from tqdm import tqdm
import pandas as pd
import time
import os 
import seaborn as sns
import random
import PCGrad_tf2 as PCGrad

start_time = time.time()

# Fix random seeds for reproducibility
np.random.seed(1234)

In [2]:
print (tf.__version__)

2.6.0


### Define PINN neural network

In [3]:
def init_model(number_hidden_layers = 2, num_neurons_per_layer = 50):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(1))
    
    model.add(tf.keras.layers.Dense(num_neurons_per_layer, activation=tf.keras.activations.get('tanh'), 
                                    kernel_initializer='glorot_normal'))
    model.add(tf.keras.layers.Dense(num_neurons_per_layer, activation=tf.keras.activations.get('tanh'), 
                                    kernel_initializer='glorot_normal'))
    
    model.add(tf.keras.layers.Dense(3, activation=tf.keras.activations.get('softmax')))
    
    return model

In [4]:
######################################################################################
# A basic PINN Tensorflow class for solving a continuous-time in-homogeneous Markov chains
class PINN:
    # Initialize the class
    def __init__(self, X_u, Y_u, X_r, model):
        
        ######################################################################################
        # Normalization constants
        self.mu_x, self.sigma_x = X_r.mean(0), X_r.std(0)

        # Normalize inputs
        X_u = (X_u - self.mu_x)/self.sigma_x
        X_r = (X_r - self.mu_x)/self.sigma_x
        self.N_u = X_u.shape[0]
        self.N_r = X_r.shape[0]

        # Store data                
        self.X_u = X_u
        self.Y_u = Y_u
        self.X_r = X_r

        self.Xu_tf = tf.convert_to_tensor(X_u, dtype=tf.float32)
        self.Yu_tf = tf.convert_to_tensor(Y_u, dtype=tf.float32)
        self.Xr_tf = tf.convert_to_tensor(X_r, dtype=tf.float32)
        
        self.model = model
        
        self.iter = 0
        
    def get_r(self):
        with tf.GradientTape(persistent=True) as g:
            g.watch(self.Xr_tf)
            
            u = self.model(self.Xr_tf)
            
            u1 = u[:,0]
            u2 = u[:,1]
            u3 = u[:,2]
            
        u_x_1 = g.jacobian(u1, self.Xr_tf)[0]/self.sigma_x
        u_x_2 = g.jacobian(u2, self.Xr_tf)[0]/self.sigma_x
        u_x_3 = g.jacobian(u3, self.Xr_tf)[0]/self.sigma_x
        
        #calculate residuals
        residual_1 = u_x_1-(-1.286e-4*u1)
        residual_2 = u_x_2-(5.6e-5*u1-1.006e-4*u2)
        residual_3 = u_x_3-(7.26e-5*u1+1.006e-4*u2)
        
        #total residual
        residual = tf.reduce_mean(tf.square(residual_1)) + tf.reduce_mean(tf.square(residual_2)) + \
        tf.reduce_mean(tf.square(residual_3))
        
        loss_1 = tf.reduce_mean(tf.square(residual_1))
        loss_2 = tf.reduce_mean(tf.square(residual_2))
        loss_3 = tf.reduce_mean(tf.square(residual_3))
        
        return loss_1, loss_2, loss_3
    
    def loss_fn(self, weight = 1):
        u_pred = self.model(self.Xu_tf)     

        # Evaluate loss
        loss_u = tf.reduce_mean(tf.square(self.Yu_tf[:, 0] - u_pred[:, 0]))
        loss_1, loss_2, loss_3 = self.get_r()
        
        return [loss_u, loss_1 + loss_2 + loss_3]
    
    def get_grad(self):
        with tf.GradientTape() as tape:
            # This tape is for derivatives with
            # respect to trainable variables
            #tape.watch(self.model.trainable_variables)
            loss = self.loss_fn()
            
        g = tape.gradient(loss, self.model.trainable_variables)
        del tape
        
        return loss, g
    
    def callback(self):
        if self.iter % 50 == 0:
            print('It {:05d}: loss = {}'.format(self.iter, ','.join(map(str, self.current_loss))))
        self.iter += 1
    
    def train(self, N, optimizer):
        """This method performs a gradient descent type optimization."""
        
        @tf.function
        def train_step():
            loss, grad_theta = self.get_grad()
            print (loss, grad_theta)
            
            # Perform gradient descent step
            optimizer.apply_gradients(zip(grad_theta, self.model.trainable_variables))
            return loss
        
        for i in range(N):
            
            loss = train_step()
            
            self.current_loss = loss
            self.callback()

In [5]:
######################################################################################
# Number of training data
N_u = 1                          # Boundary condition data on u(x)  
N_r = 500                     # Number of collocation points for minimizing the PDE residual
lb  = np.array([0.0])            # Left boundary of the domain
ub  = np.array([60000.0])        # Right boundary of the domain

# Generate training data
x_u = np.array([[0]])  ##TZ
y_u = np.array([[1,0,0]])   ##TZ                    # Solution at boundary points (dimension N_u x 1)
x_r = np.linspace(lb, ub, N_r)     # Location of collocation points (dimension N_r x 1)

In [6]:
######################################################################################
# Test data for validating the model predictions
n_star = 5000+1
x_star = np.linspace(lb, ub, n_star) #N_star = x_star.shape[0] 

In [12]:
# Compute per-task gradients.
with tf.GradientTape(persistent=True) as tape:
    model = init_model()

    PINN_solver = PINN(x_u, y_u, x_r, model)

    loss, grad_theta = PINN_solver.get_grad()
    
    assert type(loss) is list
    num_tasks = len(loss)
    loss = tf.stack(loss)
    tf.random.shuffle(loss)
    
    grads_task = tf.vectorized_map(lambda x: tf.concat([tf.reshape(grad, [-1,]) 
                        for grad in tape.gradient(x, PINN_solver.model.trainable_variables)
                        if grad is not None], axis=0), loss)
    
    # Compute gradient projections.
    def proj_grad(grad_task):
        for k in range(num_tasks):
            inner_product = tf.reduce_sum(grad_task*grads_task[k])
            proj_direction = inner_product / tf.reduce_sum(grads_task[k]*grads_task[k])
            grad_task = grad_task - tf.minimum(proj_direction, 0.) * grads_task[k]
        return grad_task

    proj_grads_flatten = tf.vectorized_map(proj_grad, grads_task)

    # Unpack flattened projected gradients back to their original shapes.
    proj_grads = []
    for j in range(num_tasks):
        start_idx = 0
        for idx, var in enumerate(PINN_solver.model.trainable_variables):
            grad_shape = var.get_shape()
            flatten_dim = np.prod([grad_shape.dims[i].value for i in range(len(grad_shape.dims))])
            proj_grad = proj_grads_flatten[j][start_idx:start_idx+flatten_dim]
            proj_grad = tf.reshape(proj_grad, grad_shape)
            if len(proj_grads) < len(PINN_solver.model.trainable_variables):
                proj_grads.append(proj_grad)
            else:
                proj_grads[idx] += proj_grad               
            start_idx += flatten_dim

    grads_and_vars = list(zip(proj_grads, PINN_solver.model.trainable_variables))

In [13]:
grads_and_vars

[(<tf.Tensor: shape=(1, 50), dtype=float32, numpy=
  array([[ 0.06028628,  0.07499516,  0.21136032,  0.1017174 ,  0.10001646,
           0.2180185 ,  0.15797585, -0.07003554, -0.00389981,  0.01137009,
          -0.02241446, -0.09091282,  0.05556903,  0.10871845, -0.1239192 ,
           0.01105038,  0.03507635, -0.19280973, -0.05704638,  0.0355982 ,
           0.19094542, -0.22936013,  0.05067914, -0.11530754, -0.06230346,
           0.02353636,  0.10330629, -0.02671034,  0.11104937, -0.17702961,
           0.01933922, -0.04847102,  0.13104624,  0.05552543, -0.08177055,
           0.0620217 , -0.00392139,  0.06948322, -0.04809517,  0.11449086,
          -0.02711431,  0.10394347, -0.08001107, -0.02420769,  0.09115085,
           0.01744652,  0.04111838,  0.05131472, -0.01880261,  0.01556524]],
        dtype=float32)>,
  <tf.Variable 'dense_9/kernel:0' shape=(1, 50) dtype=float32, numpy=
  array([[ 0.18140835,  0.05257547,  0.02610554,  0.00804662, -0.10092922,
          -0.00568269, -0.1

In [ ]:
#optim = tf.keras.optimizers.Adam()
optim = PCGrad(tf.keras.optimizers.Adam())
#PINN_solver.train(N=40000, optimizer=optim)

In [ ]:
sns.set(font_scale=1.6)